In [1]:
# import pakietów
import pandas as pd
import urllib
from IPython.display import HTML
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import re
from statistics import mean

In [2]:
#import danych
col_names = ['ID', 'gender', 'city', 'income', 'birth_date', 'application_date', 'requested_amount', 
             'requested_period', 'financial_obligations', 'employer_name', 'account_bank', 'mobile_verification_flag',
             'var_5', 'var_1', 'offer_amount', 'offer_period', 'interest_rate', 'fee', 'offer_monthly_obligation',
             'filled_form_flag', 'device', 'var_2', 'source', 'var_4', 'logged_in_flag', 'disbursed_flag']
dataset = pd.read_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/dane/train.csv', delimiter = ',', dtype='str', 
                      header = None, names = col_names, engine='python', index_col = False, skiprows = 1 )

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag
0,ID000002C20,Female,Delhi,20000,23-May-78,15-May-15,300000,5,0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000,07-Oct-85,04-May-15,200000,2,0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500,10-Oct-81,19-May-15,600000,4,0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000,30-Nov-87,09-May-15,1000000,5,0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000,17-Feb-84,20-May-15,500000,2,25000,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0


In [3]:
months_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
               'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

# TO DO - opis funckji i testy jednostkowe
def date_transform(date:str, twentieth_cent=False):
    day = date[:2]
    month = months_dict[date[3:6]]
    if twentieth_cent:
        return '19' + date[-2:] + "-" + month + "-" + day
    else:
        return '20' + date[-2:] + "-" + month + "-" + day

# formatowanie danych
dataset['income'] = dataset['income'].astype(float)
dataset['birth_date'] = dataset.apply(lambda x: date_transform(x['birth_date'], True), axis=1)
dataset['application_date'] = dataset.apply(lambda x: date_transform(x['application_date']), axis=1)
dataset['requested_amount'] = dataset['requested_amount'].astype(float)
dataset['financial_obligations'] = dataset['financial_obligations'].astype(float)

HTML(dataset.head().to_html())

,ID,gender,city,income,birth_date,application_date,requested_amount,requested_period,financial_obligations,employer_name,account_bank,mobile_verification_flag,var_5,var_1,offer_amount,offer_period,interest_rate,fee,offer_monthly_obligation,filled_form_flag,device,var_2,source,var_4,logged_in_flag,disbursed_flag
0,ID000002C20,Female,Delhi,20000.0,1978-05-23,2015-05-15,300000.0,5,0.0,CYBOSOL,HDFC Bank,N,0,HBXX,nan,nan,nan,nan,nan,N,Web-browser,G,S122,1,0,0
1,ID000004E40,Male,Mumbai,35000.0,1985-10-07,2015-05-04,200000.0,2,0.0,TATA CONSULTANCY SERVICES LTD (TCS),ICICI Bank,Y,13,HBXA,200000,2,13.25,nan,6762.9,N,Web-browser,G,S122,3,0,0
2,ID000007H20,Male,Panchkula,22500.0,1981-10-10,2015-05-19,600000.0,4,0.0,ALCHEMIST HOSPITALS LTD,State Bank of India,Y,0,HBXX,450000,4,nan,nan,nan,N,Web-browser,B,S143,1,0,0
3,ID000008I30,Male,Saharsa,35000.0,1987-11-30,2015-05-09,1000000.0,5,0.0,BIHAR GOVERNMENT,State Bank of India,Y,10,HBXX,920000,5,nan,nan,nan,N,Web-browser,B,S143,3,0,0
4,ID000009J40,Male,Bengaluru,100000.0,1984-02-17,2015-05-20,500000.0,2,25000.0,GLOBAL EDGE SOFTWARE,HDFC Bank,Y,17,HBXX,500000,2,nan,nan,nan,N,Web-browser,B,S134,3,1,0


In [4]:
# gps coordinates of cities from Internet
cities = set(dataset['city'])

In [17]:
def get_coords_from_pattern(string, pattern):
    degs = float(coords[0].split(pattern[6])[0])
    mins = float(coords[1].split(pattern[16])[0])
    secs = float(coords[2].split(pattern[33])[0])
    return degs + mins/60.0 + secs/3600.0


def convert_list_to_coord(coords_list, pattern):
    n, e = None, None
    found_n, found_e = [], []
    # wiki pattern
    if 'nsew' not in pattern:
        found_n = [float(item.split(" ")[0].split('°')[0]) for item in found]
        found_e = [float(item.split(" ")[1].split('°')[0]) for item in found]
    # google patterns
    else:
        found_n = [get_coord_from_pattern(item, pattern) for item in found if item[-1]=='n']
        found_e = [get_coord_from_pattern(item, pattern) for item in found if item[-1]=='e']   
    if found_n:
        n = mean(found_n)
    if found_e:
        e = mean(found_e)               
    return [n, e]
        
# 1 - data directly from Wikipedia

def get_wikipedia_data(city):
    pattern = "[0-9]+.[0-9]+°n [0-9]+.[0-9]+°e"
    found = []
    try:
        with urlopen("https://en.wikipedia.org/wiki/" + city) as page:
            soup = BeautifulSoup(page.read(), "html.parser")
            response_list = soup.select(".geo-dec")            
            for item in response_list:
                found += re.findall(pattern, item.get_text().lower())
    except urllib.request.HTTPError:
        pass
    return convert_list_to_coord(found, pattern)





In [45]:
# 2 - data from google search results

def get_coord_from_pattern(string, pattern):
    coords = string.split(" ")
    degs = float(coords[0].split(pattern[6])[0])
    mins = float(coords[1].split(pattern[16])[0])
    secs = float(coords[2].split(pattern[33])[0])
    return degs + mins/60.0 + secs/3600.0
    
def get_google_search_data(city):
    pattern = "[0-9]+°[ ][0-9]+'[ ][0-9]+.[0-9]+''[ ][nsew]"
    driver.get('https://www.google.pl/search?q=' + city + '+latitude+and+longitude+in+decimal')
    n,e = None, None
    found = re.findall(pattern, driver.page_source.lower())
    found_n = [get_coord_from_pattern(item, pattern) for item in found if item[-1]=='n']
    found_e = [get_coord_from_pattern(item, pattern) for item in found if item[-1]=='e']   
    if found_n:
        n = mean(found_n)
    if found_e:
        e = mean(found_e)
    return [n, e]

In [47]:
def get_latitude_to_data(city):
    pattern = "[0-9]+°[ ][0-9]+'[ ][0-9]+.[0-9]+\"[ ][nsew]"
    driver.get('https://www.google.pl/search?q=' + city + '+india+gps+coords')
    n,e = None, None
    try:
        links = driver.find_elements_by_css_selector('.rc')
        links = [item for item in links if 'latitude.to' in item.text.lower()]
        if links:
            links[0].find_element_by_css_selector('h3').click()
            undone = True
            while undone:
                if 'google' not in driver.current_url:
                    try:
                        found = re.findall(pattern, driver.page_source.lower())
                        found_n = [get_coord_from_pattern(item, pattern) for item in found if item[-1]=='n']
                        found_e = [get_coord_from_pattern(item, pattern) for item in found if item[-1]=='e']   
                        if found_n:
                            n = mean(found_n)
                        if found_e:
                            e = mean(found_e)
                        return [n, e]
                    except NoSuchElementException:
                        pass                        
                    undone = False
    except NoSuchElementException:
        pass


[31.330269444444443, 76.75663055555556]
[20.5, 85.83]
[27.6, 80.33999999999999]
[23.25, 87.85]
[20.123450000000002, 84.36906111111111]


In [7]:
# gps coordinates of cities from Internet
cities = set(dataset['city'])


    
    



def split_google_coords(str):
    city = str.split(',')[0]
    str = str.split('\n')
    longitude, latitude, dms_lat, dms_long = None, None, None, None
    for item in str:
        if 'longitude ' in item:
            longitude = item.split('longitude')[1]
        if 'latitude ' in item:
            latitude = item.split('latitude')[1]
        if 'dms lat ' in item:
            dms_lat = item.split('dms lat ')[1]
        if 'dms long ' in item:
            dms_long = item.split('dms long ')[1]
    return [city, longitude, latitude, dms_lat, dms_long]

    
driver = webdriver.Chrome('chromedriver')
cities_done = []
for city in list(cities):
    page_wiki = ''
    page_google = ''
    page_google_first_link = ''
    
    #wikipedia
    try:
        with urlopen("https://en.wikipedia.org/wiki/" + city) as page:
            soup = BeautifulSoup(page.read(), "html.parser")
            response_list = soup.select(".geo-dec")
            if response_list:
                page_wiki = soup.select(".geo-dec")[0].get_text().lower()
    except urllib.request.HTTPError:
        pass
    
    #google
    driver.get('https://www.google.pl/search?q=' + city + '+latitude+and+longitude+in+decimal')
    selector = 'div.mod'
    try:
        result = driver.find_element_by_css_selector(selector).text.lower()
        if 'india' in  result:
            page_google = split_google_coords(result)
    except NoSuchElementException:
        pass
    
    #google first returned link
    driver.get('https://www.google.pl/search?q=' + city + '+india+gps+coords')
    selector = 'h3'
    try:
        links = driver.find_elements_by_css_selector(selector)
        links = [(item, item.text.lower()) for item in links]
        links = [item for item in links if 'latitude:' in item[1] or 'longitude:' in item[1]]
        if links:
            links[0][0].click()
            undone = True
            while undone:
                if 'google' not in driver.current_url:
                    try:
                        page_google_first_link = driver.find_element_by_name('DD').get_attribute('value')
                    except NoSuchElementException:
                        pass                        
                    undone = False
    except NoSuchElementException:
        pass
    cities_done.append([city, page_wiki, page_google, page_google_first_link])
    
driver.close()
cities_coords_raw_data = pd.DataFrame(cities_done, columns = ['city', 'page_wiki', 'page_google', 'page_google_first_link'])
cities_coords_raw_data.to_csv('C:/Users/Ja/Desktop/projekt zaliczeniowy/all_coords.csv', index=False)

In [63]:
def wiki2coords(text):
    if text:
        coords = text.split(' ')
        coords[0] = float(coords[0].split('°')[0])
        coords[1] = float(coords[1].split('°')[0])
        return [coords[0], coords[1]]
    else:
        return [None, None]
       
wiki = list(cities_coords_raw_data['page_wiki'])
wiki = [wiki2coords(item) for item in wiki]

In [65]:
google = list(cities_coords_raw_data['page_google'])
pattern = "[0-9]+° [0-9]+' [0-9]+.[0-9]+'' [nsew]"

def google2coords(list):
    if list is not None:
        results = []
        for item in list:
            if item is not None:
                found = re.findall(pattern, item)
                results += found
        n, e = None, None
        if results:
            results = [(int(x.split('°')[0]), int(x.split(' ')[1].split("'")[0]), float(x.split(' ')[2].split("'")[0]), x.split(' ')[3]) for x in results]
            n = [x[0] + x[1]/60.0 + x[2]/3600.0 for x in results if x[3]=='n']
            e = [x[0] + x[1]/60.0 + x[2]/3600.0 for x in results if x[3]=='e']
            n = mean(n) if n else None
            e = mean(e) if e else None
        return [n, e]
    else:
        return None

google = [google2coords(item) for item in google]

In [71]:
google_first_link = list(cities_coords_raw_data['page_google_first_link'])
google_first_link = [ [float(x.split(' ')[0]), float(x.split(' ')[1])] if x else [None, None] for x in google_first_link]

In [94]:
# cities_coords_raw_data['wiki_lat'] = pd.Series([x[0] for x in wiki])
# cities_coords_raw_data['wiki_lon'] = pd.Series([x[1] for x in wiki])
# cities_coords_raw_data['google_lat'] = pd.Series([x[0] for x in google])
# cities_coords_raw_data['google_lon'] = pd.Series([x[1] for x in google])
# cities_coords_raw_data['google_first_link_lat'] = pd.Series([x[0] for x in google_first_link])
# cities_coords_raw_data['google_first_link_lon'] = pd.Series([x[1] for x in google_first_link])

def count_mean(list):
    list_filtered = [item for item in list if item > 0]
    if list_filtered:
        return mean(list_filtered)
    else:
        return None
    
cities_coords_raw_data['lon'] = cities_coords_raw_data.apply(lambda x: count_mean( [x['wiki_lon'], x['google_lon'], x['google_first_link_lon']]), axis = 1)
cities_coords_raw_data['lat'] = cities_coords_raw_data.apply(lambda x: count_mean( [x['wiki_lat'], x['google_lat'], x['google_first_link_lat']]), axis = 1)


,city,page_wiki,page_google,page_google_first_link,wiki_lat,wiki_lon,google_lat,google_lon,google_first_link_lat,google_first_link_lon,lat,lon
0,BAJWA,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,ADIPUR,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Chinnamiram,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,Andman & Nicobar,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,Baksa,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,SAYAN,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,BHILAD,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,Dadra & nagar Haveli,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,Rudrapur,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620,DHANGARDHA,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
cities_coords_raw_data.head()

,city,page_wiki,page_google,page_google_first_link,wiki_lat,wiki_lon,google_lat,google_lon,google_first_link_lat,google_first_link_lon,lat,lon
0,BAJWA,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mokokchung,26.32°n 94.50°e,,26.33 94.53,26.32,94.50,NaN,NaN,26.3300,94.53000,26.325000,94.515000
2,Palakkad,10.7700°n 76.6500°e,"[the latitude of palakkad, is 76.653145. pala...",10.7744 76.65625,10.77,76.65,10.784703,76.653145,10.7744,76.65625,10.776368,76.653132
3,Adilabad,19.67000°n 78.53000°e,,19.67 78.53,19.67,78.53,NaN,NaN,19.6700,78.53000,19.670000,78.530000
4,Koppal,15.35°n 76.15°e,"[the latitude of koppal, 76.155434, 15.35070...",15.35 76.15,15.35,76.15,15.350708,76.155434,15.3500,76.15000,15.350236,76.151811
